In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [9]:

# Load the dfset
df = pd.read_csv("files/input_final.csv")  # Replace with actual filename
df.head(2)



,PID,prompt_length,complexity,data_type,module,num_tokens,num_sentences,lexical_diversity,readability,noun_ratio,...,num_named_entities,contains_table,contains_list,redundancy_score,compression_ratio,contains_numbers,contains_chain_of_thought,contains_output_constraints,is_multi_turn,LLM
0,0,60,Simple,"['Text', 'Numerical Data']",document_extraction,11,1,0.909091,8.4,0.272727,...,1,False,False,1,5.454545,False,False,False,False,GPT
1,1,55,Simple,['Tables'],document_extraction,10,1,1.000000,9.2,0.200000,...,0,False,False,0,5.500000,False,False,False,False,GPT


In [10]:
df.value_counts()

PID  prompt_length  complexity  data_type                              module                        num_tokens  num_sentences  lexical_diversity  readability  noun_ratio  verb_ratio  adjective_ratio  num_named_entities  contains_table  contains_list  redundancy_score  compression_ratio  contains_numbers  contains_chain_of_thought  contains_output_constraints  is_multi_turn  LLM   
506  91             Complex     Text                                   multi_document_summarization  17          1              1.000000           10.3         0.176471    0.117647    0.176471         1                   False           False          1                 5.352941           True              False                      False                        False          GPT       1
0    60             Simple      ['Text', 'Numerical Data']             document_extraction           11          1              0.909091           8.4          0.272727    0.181818    0.000000         1                   Fals

#### drop PIP 

In [11]:
# Drop PID column as it's just an identifier
df = df.drop(columns=["PID","data_type"])
# dropping data_type column as it is not needed for training. 
df.head()

,prompt_length,complexity,module,num_tokens,num_sentences,lexical_diversity,readability,noun_ratio,verb_ratio,adjective_ratio,num_named_entities,contains_table,contains_list,redundancy_score,compression_ratio,contains_numbers,contains_chain_of_thought,contains_output_constraints,is_multi_turn,LLM
0,60,Simple,document_extraction,11,1,0.909091,8.4,0.272727,0.181818,0.000000,1,False,False,1,5.454545,False,False,False,False,GPT
1,55,Simple,document_extraction,10,1,1.000000,9.2,0.200000,0.200000,0.000000,0,False,False,0,5.500000,False,False,False,False,GPT
2,95,Simple,document_extraction,17,1,0.941176,10.3,0.235294,0.176471,0.000000,0,False,False,1,5.588235,False,False,False,False,GPT
3,97,Simple,document_extraction,19,1,0.842105,9.5,0.315789,0.105263,0.052632,0,False,False,2,5.105263,False,False,False,False,GPT
4,71,Simple,document_extraction,13,1,1.000000,9.9,0.153846,0.153846,0.076923,1,False,False,1,5.461538,False,False,False,False,GPT


##### Encode the target variable (LLM)


In [12]:
le = LabelEncoder()
df["LLM"] = le.fit_transform(df["LLM"])
df.head()


,prompt_length,complexity,module,num_tokens,num_sentences,lexical_diversity,readability,noun_ratio,verb_ratio,adjective_ratio,num_named_entities,contains_table,contains_list,redundancy_score,compression_ratio,contains_numbers,contains_chain_of_thought,contains_output_constraints,is_multi_turn,LLM
0,60,Simple,document_extraction,11,1,0.909091,8.4,0.272727,0.181818,0.000000,1,False,False,1,5.454545,False,False,False,False,1
1,55,Simple,document_extraction,10,1,1.000000,9.2,0.200000,0.200000,0.000000,0,False,False,0,5.500000,False,False,False,False,1
2,95,Simple,document_extraction,17,1,0.941176,10.3,0.235294,0.176471,0.000000,0,False,False,1,5.588235,False,False,False,False,1
3,97,Simple,document_extraction,19,1,0.842105,9.5,0.315789,0.105263,0.052632,0,False,False,2,5.105263,False,False,False,False,1
4,71,Simple,document_extraction,13,1,1.000000,9.9,0.153846,0.153846,0.076923,1,False,False,1,5.461538,False,False,False,False,1


#### Separate features and target



In [13]:
y = df["LLM"]
X = df.drop(columns=["LLM"])

print(X.columns)

Index(['prompt_length', 'complexity', 'module', 'num_tokens', 'num_sentences',
       'lexical_diversity', 'readability', 'noun_ratio', 'verb_ratio',
       'adjective_ratio', 'num_named_entities', 'contains_table',
       'contains_list', 'redundancy_score', 'compression_ratio',
       'contains_numbers', 'contains_chain_of_thought',
       'contains_output_constraints', 'is_multi_turn'],
      dtype='object')


##### one-hot encoding of categorical features

In [8]:
# get categorical columns
cat_columns = X.select_dtypes(['object']).columns
cat_columns

Index(['complexity', 'module'], dtype='object')

In [14]:
categorical_cols = ["complexity", "module"]  # Exclude "LLM"
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

# Fit and transform only the selected categorical columns
categorical_encoded = ohe.fit_transform(X[categorical_cols])

# Convert to DataFrame
categorical_df = pd.DataFrame(categorical_encoded, columns=ohe.get_feature_names_out(categorical_cols))

categorical_df.head()
X = pd.concat([X, categorical_df], axis=1)
X = X.drop(columns=categorical_cols)

X.head()


,prompt_length,num_tokens,num_sentences,lexical_diversity,readability,noun_ratio,verb_ratio,adjective_ratio,num_named_entities,contains_table,...,contains_output_constraints,is_multi_turn,complexity_Complex,complexity_Medium,complexity_Simple,module_document_extraction,module_interactive_qa_chatbot,module_multi_document_summarization,module_quantitative_analysis,module_report_generation
0,60,11,1,0.909091,8.4,0.272727,0.181818,0.000000,1,False,...,False,False,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,55,10,1,1.000000,9.2,0.200000,0.200000,0.000000,0,False,...,False,False,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,95,17,1,0.941176,10.3,0.235294,0.176471,0.000000,0,False,...,False,False,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,97,19,1,0.842105,9.5,0.315789,0.105263,0.052632,0,False,...,False,False,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,71,13,1,1.000000,9.9,0.153846,0.153846,0.076923,1,False,...,False,False,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [15]:
X.columns

Index(['prompt_length', 'num_tokens', 'num_sentences', 'lexical_diversity',
       'readability', 'noun_ratio', 'verb_ratio', 'adjective_ratio',
       'num_named_entities', 'contains_table', 'contains_list',
       'redundancy_score', 'compression_ratio', 'contains_numbers',
       'contains_chain_of_thought', 'contains_output_constraints',
       'is_multi_turn', 'complexity_Complex', 'complexity_Medium',
       'complexity_Simple', 'module_document_extraction',
       'module_interactive_qa_chatbot', 'module_multi_document_summarization',
       'module_quantitative_analysis', 'module_report_generation'],
      dtype='object')

#### Test Train split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

In [ ]:
# save subset for later use
subset = X.head(4)
subset.to_csv("files/X_train_subset.csv", index=False)


In [142]:
X_train.head()

,prompt_length,num_tokens,num_sentences,lexical_diversity,readability,noun_ratio,verb_ratio,adjective_ratio,num_named_entities,contains_table,...,contains_output_constraints,is_multi_turn,complexity_Complex,complexity_Medium,complexity_Simple,module_document_extraction,module_interactive_qa_chatbot,module_multi_document_summarization,module_quantitative_analysis,module_report_generation
475,90,14,1,1.000000,13.9,0.285714,0.142857,0.214286,0,False,...,False,False,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
499,85,14,1,1.000000,16.2,0.214286,0.142857,0.142857,1,False,...,False,False,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
410,64,12,1,0.916667,7.6,0.250000,0.083333,0.083333,0,False,...,False,False,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
273,52,9,1,1.000000,8.8,0.444444,0.111111,0.111111,1,False,...,False,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
291,69,13,1,0.846154,9.2,0.230769,0.153846,0.000000,0,False,...,False,False,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


#### Feature scaling

In [19]:

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



#### Train the classifier


In [18]:
# !pip install xgboost

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

from xgboost import XGBClassifier

param_grid = {
    "n_estimators": [50, 100, 200],  # Number of boosting rounds
    "learning_rate": [0.01, 0.1, 0.2],  # Step size shrinkage
    "max_depth": [3, 6, 10],  # Maximum depth of a tree
    "min_child_weight": [1, 3, 5],  # Minimum sum of instance weight needed in a child
    "subsample": [0.8, 1.0],  # Fraction of samples used for training
    "colsample_bytree": [0.8, 1.0],  # Fraction of features used per tree
}

xgb = XGBClassifier(objective="multi:softmax", num_class=len(le.classes_), random_state=42, use_label_encoder=False)
# Define the hyperparameter grid for XGBoost

# Perform Grid Search
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    n_jobs=-1,  # Use all available processors
    verbose=2,
    scoring="accuracy"
)

grid_search.fit(X_train, y_train)

# Get the best model
best_xgb = grid_search.best_estimator_

# Predict on test data
y_pred = best_xgb.predict(X_test)

##### XGB model eval

In [26]:
# Evaluate the model
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Convert integer labels back to class names
y_test_labels = le.inverse_transform(y_test)  # Convert encoded labels to original class names
y_pred_labels = le.inverse_transform(y_pred)  # Convert predicted labels to original class names

print(classification_report(y_test_labels, y_pred_labels, target_names=le.classes_))
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
precision_score = precision_score(y_test, y_pred, average='weighted')
print(f"Precision: {precision_score:.2f}")
recall_score = recall_score(y_test, y_pred, average='weighted')
print(f"Recall: {recall_score:.2f}")
f1_score = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Score: {f1_score:.2f}")

              precision    recall  f1-score   support

      Claude       0.67      0.50      0.57         8
         GPT       0.82      0.82      0.82        11
     Mistral       0.75      0.75      0.75        16
        llma       0.78      0.88      0.82        16

    accuracy                           0.76        51
   macro avg       0.75      0.74      0.74        51
weighted avg       0.76      0.76      0.76        51

Accuracy: 0.76
Precision: 0.76
Recall: 0.76
F1 Score: 0.76


['files/xgboost.pkl']

##### GridsearchCV - Random forest classifier

In [27]:

# Define the parameter grid
param_grid = {
    "n_estimators": [50, 100, 200],  # Number of trees in the forest
    "max_depth": [10, 20, None],     # Maximum depth of the trees
    "min_samples_split": [2, 5, 10], # Minimum number of samples required to split an internal node
    "min_samples_leaf": [1, 2, 4]    # Minimum number of samples required to be at a leaf node
}

# Initialize the model
rf = RandomForestClassifier(random_state=42)

# Perform Grid Search
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    n_jobs=-1,  # Use all available processors
    verbose=2,
    scoring="accuracy"
)

grid_search.fit(X_train, y_train)

# Best model from GridSearchCV
best_rf = grid_search.best_estimator_

# Evaluate the optimized model
y_pred = best_rf.predict(X_test)


y_pred = best_rf.predict(X_test)
y_pred

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s

[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.1s
[CV] END max_depth=10, min_samples_lea

array([0, 1, 1, 3, 2, 3, 3, 2, 0, 3, 0, 3, 3, 0, 3, 2, 2, 2, 2, 3, 2, 2,
       2, 3, 3, 0, 2, 3, 3, 3, 3, 2, 1, 3, 2, 2, 2, 2, 1, 1, 1, 3, 1, 1,
       3, 0, 2, 3, 3, 1, 1])

##### Model evaluation Random forest classifier

In [29]:
# Evaluate the model
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Convert integer labels back to class names
y_test_labels = le.inverse_transform(y_test)  # Convert encoded labels to original class names
y_pred_labels = le.inverse_transform(y_pred)  # Convert predicted labels to original class names

print(classification_report(y_test_labels, y_pred_labels, target_names=le.classes_))
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
precision_score = precision_score(y_test, y_pred, average='weighted')
print(f"Precision: {precision_score:.2f}")
recall_score = recall_score(y_test, y_pred, average='weighted')
print(f"Recall: {recall_score:.2f}")
f1_score = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Score: {f1_score:.2f}")

              precision    recall  f1-score   support

      Claude       0.67      0.50      0.57         8
         GPT       0.80      0.73      0.76        11
     Mistral       0.75      0.75      0.75        16
        llma       0.74      0.88      0.80        16

    accuracy                           0.75        51
   macro avg       0.74      0.71      0.72        51
weighted avg       0.74      0.75      0.74        51

Accuracy: 0.75
Precision: 0.74
Recall: 0.75
F1 Score: 0.74


In [32]:
import joblib

# Save the MultiLabelBinarizer model
joblib.dump(best_rf, "files/best_random_forest.pkl")

joblib.dump(best_xgb, "files/best_xgboost.pkl") 

# Save the LabelEncoder
joblib.dump(le, "files/v5_random_forest_label_encoder.pkl")

# Save the StandardScaler
joblib.dump(scaler, "files/v5_random_forest_scaler.pkl")

# Save the OneHotEncoder
joblib.dump(ohe, "files/v5_random_forest_onehot_encoder.pkl")


['files/v5_random_forest_onehot_encoder.pkl']

##### Inference

##### Extract features

In [31]:
import spacy
import textstat
import re
from collections import Counter

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

def extract_prompt_features(prompt):
    doc = nlp(prompt)
    
    #  Linguistic Features
    num_tokens = len(doc)
    num_sentences = len(list(doc.sents))
    unique_words = len(set([token.text.lower() for token in doc]))
    lexical_diversity = unique_words / num_tokens if num_tokens > 0 else 0
    readability = textstat.flesch_kincaid_grade(prompt)
    
    # Part-of-Speech (POS) Distribution
    pos_counts = Counter([token.pos_ for token in doc])
    noun_ratio = pos_counts.get("NOUN", 0) / num_tokens
    verb_ratio = pos_counts.get("VERB", 0) / num_tokens
    adjective_ratio = pos_counts.get("ADJ", 0) / num_tokens

    #  Structural Features
    num_named_entities = len(doc.ents)
    contains_table = bool(re.search(r"(\||\+\-+)|Table \d+", prompt))  # Detect tables
    contains_list = bool(re.search(r"(\d+\.)|(- )|(\* )", prompt))  # Detect bullet points or numbered lists

    # Content-Specific Features
    keyword_density = {word: prompt.lower().count(word) for word in ["summary", "analyze", "data", "report"]}
    redundancy_score = len(re.findall(r"(\b\w+\b).*\1", prompt))  # Count repeated words
    compression_ratio = len(prompt) / num_tokens if num_tokens > 0 else 0

    #  Task-Specific Complexity Features
    contains_numbers = bool(re.search(r"\d+", prompt))  # Presence of numerical data
    contains_chain_of_thought = bool(re.search(r"step-by-step|explain your reasoning", prompt.lower()))
    contains_output_constraints = bool(re.search(r"limit to|output in|return a json", prompt.lower()))
    is_multi_turn = "previous response" in prompt.lower() or "as mentioned before" in prompt.lower()

    return {
        "num_tokens": num_tokens,
        "num_sentences": num_sentences,
        "lexical_diversity": lexical_diversity,
        "readability": readability,
        "noun_ratio": noun_ratio,
        "verb_ratio": verb_ratio,
        "adjective_ratio": adjective_ratio,
        "num_named_entities": num_named_entities,
        "contains_table": contains_table,
        "contains_list": contains_list,
        "redundancy_score": redundancy_score,
        "compression_ratio": compression_ratio,
        "contains_numbers": contains_numbers,
        "contains_chain_of_thought": contains_chain_of_thought,
        "contains_output_constraints": contains_output_constraints,
        "is_multi_turn": is_multi_turn
    }




##### Load model

In [33]:

import joblib
# load the model and use it to make new predictions
model = joblib.load("files/best_xgboost.pkl")
scaler = joblib.load("files/v5_random_forest_scaler.pkl")
ohe = joblib.load("files/v5_random_forest_onehot_encoder.pkl")
le = joblib.load("files/v5_random_forest_label_encoder.pkl")


def classify_complexity(prompt, num_tokens=25, readability=70, num_sentences=1, lexical_diversity=2, num_named_entities=3):
    if num_tokens <= 10 and readability >= 60 and num_sentences == 1:
        return "Simple"
    elif 10 < num_tokens <= 20 or (40 <= readability < 60) or (num_named_entities <= 4):
        return "Medium"
    else:
        return "Complex"


print()  # Output: Medium


In [ ]:
# write a prompt for summarizing a text


In [35]:
import pandas as pd
# Convert input into DataFrame
# new_df = pd.DataFrame([new_input])
# Example Prompt
prompt = """
You are an AI assistant. Summarize the research paper and provide key insights. 
Ensure the response is formatted as JSON and limited to 200 words. 
Step-by-step analysis is preferred. Example: "In this study, the authors examined..."
"""
prompt = "What was the main finding of the report?"
# Extract Features
features = {}
X_train_subset = pd.read_csv("files/X_train_subset.csv")

features = extract_prompt_features(prompt)
complexity = classify_complexity(prompt)
features.update({"complexity": complexity, "module": "document_extraction","prompt_length": len(prompt)})
print(features)

df = pd.DataFrame([features],index=[0])

# Transform the input using the saved encoders


categorical_cols = ["complexity", "module"]
categorical_encoded = ohe.transform(df[categorical_cols])
categorical_df = pd.DataFrame(categorical_encoded, columns=ohe.get_feature_names_out())



df = pd.concat([df, categorical_df], axis=1)
new_df = df.drop(columns=categorical_cols)

new_df.head()
new_df = new_df[X_train_subset.columns]
new_df = scaler.transform(new_df)

# # # Make a prediction
prediction = model.predict(new_df)
predicted_class = le.inverse_transform(prediction)[0]
print(f"Predicted class: {predicted_class}")


# Index(['prompt_length', 'complexity', 'module', 'num_tokens', 'num_sentences',
#        'lexical_diversity', 'readability', 'noun_ratio', 'verb_ratio',
#        'adjective_ratio', 'num_named_entities', 'contains_table',
#        'contains_list', 'redundancy_score', 'compression_ratio',
#        'contains_numbers', 'contains_chain_of_thought',
#        'contains_output_constraints', 'is_multi_turn', 'LLM'],
#       dtype='object')



{'num_tokens': 9, 'num_sentences': 1, 'lexical_diversity': 0.8888888888888888, 'readability': 2.9, 'noun_ratio': 0.2222222222222222, 'verb_ratio': 0.0, 'adjective_ratio': 0.1111111111111111, 'num_named_entities': 0, 'contains_table': False, 'contains_list': False, 'redundancy_score': 1, 'compression_ratio': 4.444444444444445, 'contains_numbers': False, 'contains_chain_of_thought': False, 'contains_output_constraints': False, 'is_multi_turn': False, 'complexity': 'Medium', 'module': 'document_extraction', 'prompt_length': 40}
Predicted class: GPT


In [36]:
prob_scores = model.predict_proba(new_df)
print(prob_scores)
# Get predicted class and confidence score
predicted_class = model.predict(new_df)[0]
confidence_score = np.max(prob_scores) * 100  # Convert to percentage

print(f"Predicted Class: {predicted_class}")
print(f"Confidence Score: {confidence_score:.2f}%")

[[0.23584124 0.33676404 0.22134596 0.20604873]]
Predicted Class: 1
Confidence Score: 33.68%
